In [38]:
import numpy as np
from pathlib import Path
from tqdm import tqdm
import pandas as pd
from tbparse import SummaryReader

pd.set_option('display.max_columns', None)

models = {
    "NeRF (sigma=10)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_10/ckpt/cf.txt",
    "NeRF (sigma=50)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_50/ckpt/cf.txt",
    "NeRF (sigma=100)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_100/ckpt/cf.txt",
    # "NeRF (sigma=200)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_200/ckpt/cf.txt",
    # "NeRF (mean 0.1)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/depth_mean_0.1/ckpt/cf.txt",
    # "NeRF (mean 0.9)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/depth_mean_0.9/ckpt/cf.txt",
    # "NeRF (mode 0.1)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/depth_mode_0.1/ckpt/cf.txt",
    # "NeRF (mode 0.9)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/depth_mode_0.9/ckpt/cf.txt",
    # "NeRF (test mean 0.1)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/depth_spiral_mean_0.1/ckpt/cf.txt",
    # "NeRF (test mode 0.1)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/depth_spiral_mode_0.1/ckpt/cf.txt",

    "Mip360 (sigma=10)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_10_pts/cf.txt",
    "Mip360 (sigma=50)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_50_pts/cf.txt",
    "Mip360 (sigma=100)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_100_pts/cf.txt",
    # "MipNeRF360 (sigma=200)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_200_pts/cf.txt",

    "NeuS White" : "/rds/project/rds-qxpdOeYWi78/NeuS/exp/{}/womask_white/fine_mesh/cf.txt",
    "NeuS" : "/rds/project/rds-qxpdOeYWi78/NeuS/exp/{}/womask/fine_mesh/cf.txt",
    # "NeuS 0.5" : "/rds/project/rds-qxpdOeYWi78/NeuS/exp/{}/womask/fine_mesh/max_dist05/cf.txt",
    "HFS White" : "/rds/project/rds-qxpdOeYWi78/HFS/exps_1.5/{}/womask_white_no_bg/fine_mesh/cf.txt",
    "HFS" : "/rds/project/rds-qxpdOeYWi78/HFS/exps_1.5/{}/womask_hfs/fine_mesh/cf.txt",
    # "HFS 0.5" : "/rds/project/rds-qxpdOeYWi78/HFS/exps_1.5/{}/womask_hfs/fine_mesh/max_dist05/cf.txt",

    # "Ours (clamp_0149/surf_masked)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/clamp_0149/ckpt_eval_surf_masked/ckpt/cf.txt",
    # "Ours (clamp_0149_no_ldist/surf_masked)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/clamp_0149_no_ldist/ckpt_eval_surf_masked/ckpt/cf.txt",
    
    # "Ours (clamp_0149_no_ldist_no_in_sparse/surf_single)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/clamp_0149_no_ldist_no_in_sparse/ckpt_eval_surf_single/ckpt/cf.txt",
    # "Ours (clamp_0149_no_ldist_no_in_sparse/surf_masked)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/clamp_0149_no_ldist_no_in_sparse/ckpt_eval_surf_masked/ckpt/cf.txt",
    # "Ours (solid_no_fs/surf_single)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/solid_no_fs/ckpt_eval_surf_single/ckpt/cf.txt",
    # "Ours (solid_no_fs/surf_masked)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/solid_no_fs/ckpt_eval_surf_masked/ckpt/cf.txt",
    # "Ours": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/solid_no_fs/ckpt_eval_surf_single/ckpt/cf.txt",
    # "Ours (trimmed)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/solid_no_fs/ckpt_eval_surf_masked/ckpt/cf.txt",

    "Ours": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/solid_less_trunc/ckpt_eval_surf_single/ckpt/cf.txt",
    "Ours (trimmed)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/solid_less_trunc/ckpt_eval_surf_masked/ckpt/cf.txt",
    
    # "Ours (clamp_0149_no_ldist_no_in_sparse/surf)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/clamp_0149_no_ldist_no_in_sparse/ckpt_eval_surf/ckpt/cf.txt",
    # "Ours (alpha_dep_tv_mlv/surf_masked)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/alpha_dep_tv_mlv/ckpt_eval_surf_masked/ckpt/cf.txt",
    # "Ours (alpha_dep_tv_mlv/surf)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/alpha_dep_tv_mlv/ckpt_eval_surf/ckpt/cf.txt",
    #
    #  "Ours (clamp_0149_mlv_2/surf_masked)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/clamp_0149_mlv_2/ckpt_eval_surf_masked/ckpt/cf.txt",
    # "Ours (clamp_0149_mlv_2/surf)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/clamp_0149_mlv_2/ckpt_eval_surf/ckpt/cf.txt",
    # "Ours (clamp_0149_mlv_2/surf_single)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/clamp_0149_mlv_2/ckpt_eval_surf_single/ckpt/cf.txt",
    # "Ours (clamp_0149_mlv_2/surf_masked)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/clamp_0149_mlv_2/ckpt_eval_surf_masked/ckpt/cf.txt",
    # "Ours (trans2_0001/surf)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/trans2_0001/ckpt_eval_surf/ckpt/cf.txt",
    # "Ours (trans2_0001/surf_single)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/trans2_0001/ckpt_eval_surf_single/ckpt/cf.txt",
    # "Ours (trans2_0001/surf_masked)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/trans2_0001/ckpt_eval_surf_masked/ckpt/cf.txt",
    # "Ours (trans2_0001_low_sparse/surf_single)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/trans2_0001_low_sparse/ckpt_eval_surf_single/ckpt/cf.txt",
    # "Ours (trans2_0001_low_sparse/surf_masked)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/trans2_0001_low_sparse/ckpt_eval_surf_masked/ckpt/cf.txt",
    # "Ours (trans2_0001_no_sparse/surf_single)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/trans2_0001_no_sparse/ckpt_eval_surf_single/ckpt/cf.txt",
    # "Ours (trans2_0001_no_sparse/surf_masked)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/trans2_0001_no_sparse/ckpt_eval_surf_masked/ckpt/cf.txt",
    # "Ours (trans2_0001_no_sparse_no_entro/surf_single)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/trans2_0001_no_sparse_no_entro/ckpt_eval_surf_single/ckpt/cf.txt",
    # "Ours (trans2_0001_no_sparse_no_entro/surf_masked)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/trans2_0001_no_sparse_no_entro/ckpt_eval_surf_masked/ckpt/cf.txt",
    # "Ours (trans_no_fs/surf_single)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/trans_no_fs/ckpt_eval_surf_single/ckpt/cf.txt",
    # "Ours (trans_no_fs/surf_masked)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/trans_no_fs/ckpt_eval_surf_masked/ckpt/cf.txt",
    
    # "Ours (trans_no_fs_no_sparse/surf_single)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/trans_no_fs_no_sparse/ckpt_eval_surf_single/ckpt/cf.txt",
    # "Ours (trans_no_fs_no_sparse/surf_masked)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/trans_no_fs_no_sparse/ckpt_eval_surf_masked/ckpt/cf.txt",
    
    # "Ours (trans_no_fs_low_sparse/surf_single)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/trans_no_fs_low_sparse/ckpt_eval_surf_single/ckpt/cf.txt",
    # "Ours (trans_no_fs_low_sparse/surf_masked)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/trans_no_fs_low_sparse/ckpt_eval_surf_masked/ckpt/cf.txt",
    # "Ours (trans_no_fs_low_sparse_2/surf_single)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/trans_no_fs_low_sparse_2/ckpt_eval_surf_single/ckpt/cf.txt",
    # "Ours (trans_no_fs_low_sparse_2/surf_masked)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/trans_no_fs_low_sparse_2/ckpt_eval_surf_masked/ckpt/cf.txt",
    # "Ourst": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/trans_no_fs_low_sparse_2/ckpt_eval_surf_single/ckpt/cf.txt",
    # "Ourst (trimmed)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/trans_no_fs_low_sparse_2/ckpt_eval_surf_masked/ckpt/cf.txt",

    

    "NeRF (depth 0.1)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/depth_spiral_mode_0.1/remeshed/cf.txt",
    "Mip360 (depth 0.9)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/distance_mode_09_pts/cf.txt",
    "Mip360 (depth 0.1)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/distance_mode_pts/cf.txt",
    
    
    # "Ours (mlv_trunc_conv_2_0028/surf_masked)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/mlv_trunc_conv_2_0028/ckpt_eval_surf_masked/ckpt/cf.txt",
    # "Ours (mlv_trunc_conv_2_0028/surf)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/mlv_trunc_conv_2_0028/ckpt_eval_surf/ckpt/cf.txt",
    # "Ours (mlv_trunc_conv_2_0028_no_ldist/surf)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/mlv_trunc_conv_2_0028_no_ldist/ckpt_eval_surf/ckpt/cf.txt",
    # "Ours (alpha_dep_tv_slv/surf)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/alpha_dep_tv_slv/ckpt_eval_surf/ckpt/cf.txt",
    # "Ours (alpha_dep_tv_slv/surf_masked)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/alpha_dep_tv_slv/ckpt_eval_surf_masked/ckpt/cf.txt",

}


def read_entry(p, scale=1e-3):
    entry = {
        "name": "",
        # "Mean d2s": None,
        # "Mean s2d": None,
        "Avg cf": None,
        # "Accuracy": None,
        # "Completeness": None,
        # "F1": None,
        # "psnr": None,
    }
    if Path(p).exists():
        with Path(p).open('r') as f:
            lines = f.readlines()

            for k in entry.keys():
                for l in lines:
                    if l.startswith(k):
                        value = l.split(':')[-1].strip()
                        if value != 'None':
                            entry[k] = float(value) 
                            if k == 'Avg cf': entry[k]/= scale
    
    return entry

def get_scene_comp(scene_name, scale=1e-3):
    df = pd.DataFrame()
    for name in models.keys():
        e = read_entry(models[name].format(scene_name), scale)
        e['name'] = name

        df = pd.concat([df, pd.DataFrame(e, index=[len(df)])])
    return df

replace_dict = {
    'ship\_re':'ship',
    'lego\_re':'lego',
    'mic\_re':'mic',
    'hotdog\_re':'hotdog',
    'chair\_re':'chair',
    '(sigma=10)':'($\sigma=10$)',
    '(sigma=50)':'($\sigma=50$)',
    '(sigma=100)':'($\sigma=100$)',
    'test mode':'depth',
    'Ours (clamp\_0149\_no\_ldist\_no\_in\_sparse/surf\_sin...':'Ours',
    'Ours (clamp\_0149\_no\_ldist\_no\_in\_sparse/surf\_mas...':'Ours (trimmed)',
    'Ours (solid\_no\_fs/surf\_single)': 'Ours',
    'Ours (solid\_no\_fs/surf\_masked)': 'Ours (trimmed)',
    'Ours (trans\_no\_fs\_low\_sparse\_2/surf\_single)': 'Ours',
    'Ours (trans\_no\_fs\_low\_sparse\_2/surf\_masked)': 'Ours (trimmed)',

}

def highlight_second_min(s):
    styles = []
    min_value = s.min()
    second_min_value = s[s > min_value].min()
    for v in s:
        if v == second_min_value:
            styles.append('background-color: violet')
        else:
            styles.append('')
    return styles

def highlight_third_min(s):
    styles = []
    min_value = s.min()
    second_min_value = s[s > min_value].min()
    third_min_value = s[s > second_min_value].min()
    for v in s:
        if v == third_min_value:
            styles.append('background-color: thistle')
        else:
            styles.append('')
    return styles

def highlight_min_values(s):
    styles = []
    sorted_s = s.sort_values()
    min_value = sorted_s.iloc[0]
    second_min_value = sorted_s.iloc[1]
    third_min_value = sorted_s.iloc[2]
    for v in s:
        if v == min_value:
            styles.append('background-color: magenta')
        elif v == second_min_value:
            styles.append('background-color: violet')
        elif v == third_min_value:
            styles.append('background-color: thistle')
        else:
            styles.append('')
    return styles

In [39]:
scene_names = [
    "dtu_scan24", 
    "dtu_scan37",
    "dtu_scan40",
    "dtu_scan55",
    "dtu_scan63", 
    "dtu_scan65",
    "dtu_scan83", 
    "dtu_scan69",
    "dtu_scan97",
    "dtu_scan105", 
    "dtu_scan106",
    "dtu_scan110",
    "dtu_scan114",
    "dtu_scan118",
    "dtu_scan122",
]



models = {
    "NeRF (sigma=10)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_10/ckpt/cf.txt",
    "NeRF (sigma=50)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_50/ckpt/cf.txt",
    "NeRF (sigma=100)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_100/ckpt/cf.txt",

    # "NeRF black (sigma=10)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn_black/ckpt_eval_cuvol/thresh_10/ckpt/cf.txt",
    # "NeRF black (sigma=50)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn_black/ckpt_eval_cuvol/thresh_50/ckpt/cf.txt",
    # "NeRF black (sigma=100)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn_black/ckpt_eval_cuvol/thresh_100/ckpt/cf.txt",

    # "Ours sparse": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/sparse/ckpt_eval_surf_single/ckpt/cf.txt",
    # "Ours sparse (trimmed)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/sparse/ckpt_eval_surf_masked/ckpt/cf.txt",
    
    # "Ours less_sparse": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_sparse/ckpt_eval_surf_single/ckpt/cf.txt",
    # "Ours less_sparse (trimmed)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_sparse/ckpt_eval_surf_masked/ckpt/cf.txt",
    
    "Ours less_trunc": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_trunc/ckpt_eval_surf_single/ckpt/cf.txt",
    "Ours less_trunc (trimmed)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_trunc/ckpt_eval_surf_masked/ckpt/cf.txt",
    
    # "Ours less_trunc_black": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_trunc_black/ckpt_eval_surf_single/ckpt/cf.txt",
    # "Ours less_trunc_black (trimmed)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_trunc_black/ckpt_eval_surf_masked/ckpt/cf.txt",
    
    # "Ours less_trunc_less_sparse": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_trunc_less_sparse/ckpt_eval_surf_single/ckpt/cf.txt",
    
    # "Ours less_trunc_2": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_trunc_2/ckpt_eval_surf_single/ckpt/cf.txt",
    # "Ours less_trunc_2 (trimmed)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_trunc_2/ckpt_eval_surf_masked/ckpt/cf.txt",

    # "NeRF (depth 0.1)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/depth_spiral_mode_0.1/remeshed/cf.txt",
}


df_all = None

digit = 2

for i, scene in enumerate(scene_names):
    df = get_scene_comp(scene, 1)
    df = df.round(digit)
    # df.loc[-1] = ["scene name", scene, scene]
    df = df.rename(columns={'Avg cf':scene})
    # df.loc[-1] = ["scene name", scene]
    # df.index = df.index + 1  # shifting index
    # df.sort_index(inplace=True)

    if i > 0:
        df = df.drop('name', axis=1)
    df_all = pd.concat([df_all, df], axis=1)

    

# neus = ['neus', 1.37, 1.21, 0.73, 0.40, 1.20, 0.70, 0.72, 1.01, 1.116, 0.82, 0.66, 1.69, 0.39, 0.49, 0.51]
# df_all.loc['NeuS'] = ['neus', 1.37, 1.21, 0.73, 0.40, 1.20, 0.70, 0.72, 1.01, 1.116, 0.82, 0.66, 1.69, 0.39, 0.49, 0.51]

# df_all['mean cf'] = df_all['Avg cf'][1:].mean(axis=1).round(digit)
df_all['avg'] = df_all[scene_names].mean(axis=1).round(digit)
# df_all['mean F1'] = df_all['F1'][1:].mean(axis=1).round(digit)


# df_all.style.highlight_min(color = 'purple', axis = 0).apply(highlight_second_min).apply(highlight_third_min)
df_all.style.apply(highlight_min_values)

,name,dtu_scan24,dtu_scan37,dtu_scan40,dtu_scan55,dtu_scan63,dtu_scan65,dtu_scan83,dtu_scan69,dtu_scan97,dtu_scan105,dtu_scan106,dtu_scan110,dtu_scan114,dtu_scan118,dtu_scan122,avg
0,NeRF (sigma=10),1.990000,1.900000,1.860000,1.710000,1.860000,1.960000,2.220000,2.040000,2.430000,1.820000,1.850000,1.960000,1.670000,1.990000,1.710000,1.930000
1,NeRF (sigma=50),1.370000,1.460000,1.430000,1.220000,1.660000,1.850000,2.030000,1.600000,1.980000,1.260000,1.490000,1.750000,1.450000,1.280000,1.210000,1.540000
2,NeRF (sigma=100),1.510000,1.340000,1.570000,0.880000,2.990000,2.000000,2.840000,2.220000,2.800000,2.080000,1.720000,2.430000,1.440000,1.910000,1.650000,1.960000
3,Ours less_trunc,1.470000,1.660000,1.590000,1.340000,1.180000,2.260000,1.530000,2.300000,3.360000,1.310000,1.830000,1.590000,1.310000,2.220000,1.450000,1.760000
4,Ours less_trunc (trimmed),1.040000,1.360000,1.300000,1.100000,0.890000,2.150000,1.290000,1.900000,2.840000,0.990000,1.710000,1.330000,1.090000,1.880000,1.240000,1.470000


In [62]:
scene_names = [
    "dtu_scan24", 
    "dtu_scan37",
    "dtu_scan40",
    "dtu_scan55",
    "dtu_scan63", 
    "dtu_scan65",
    "dtu_scan83", 
    "dtu_scan69",
    "dtu_scan97",
    "dtu_scan105", 
    "dtu_scan106",
    "dtu_scan110",
    "dtu_scan114",
    "dtu_scan118",
    "dtu_scan122",
]



models = {
    "NeRF (sigma=10)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_10/ckpt/cf.txt",
    "NeRF (sigma=50)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_50/ckpt/cf.txt",
    "NeRF (sigma=100)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_100/ckpt/cf.txt",

    # "NeRF black (sigma=10)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn_black/ckpt_eval_cuvol/thresh_10/ckpt/cf.txt",
    # "NeRF black (sigma=50)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn_black/ckpt_eval_cuvol/thresh_50/ckpt/cf.txt",
    # "NeRF black (sigma=100)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn_black/ckpt_eval_cuvol/thresh_100/ckpt/cf.txt",

    # "Ours sparse": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/sparse/ckpt_eval_surf_single/ckpt/cf.txt",
    # "Ours sparse (trimmed)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/sparse/ckpt_eval_surf_masked/ckpt/cf.txt",
    
    # "Ours less_sparse": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_sparse/ckpt_eval_surf_single/ckpt/cf.txt",
    # "Ours less_sparse (trimmed)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_sparse/ckpt_eval_surf_masked/ckpt/cf.txt",
    
#     "NeRF (depth 0.1)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/depth_spiral_mode_0.1/remeshed/cf.txt",
#     "Ours less_trunc": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_trunc/ckpt_eval_surf_single/ckpt/cf.txt",
    "Ours less_trunc all surf": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_trunc/ckpt_eval_surf_mlv/ckpt/cf.txt",
#     "Ours less_trunc (trimmed)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_trunc/ckpt_eval_surf_masked/ckpt/cf.txt",
    
#     "Ours less_trunc_norm all surf": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_trunc_norm_59/ckpt_eval_surf_mlv/ckpt/cf.txt",
    
    # "Ours less_trunc_black": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_trunc_black/ckpt_eval_surf_single/ckpt/cf.txt",
    # "Ours less_trunc_black (trimmed)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_trunc_black/ckpt_eval_surf_masked/ckpt/cf.txt",
    
    # "Ours less_trunc_less_sparse": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_trunc_less_sparse/ckpt_eval_surf_single/ckpt/cf.txt",
    
    # "Ours less_trunc_2": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_trunc_2/ckpt_eval_surf_single/ckpt/cf.txt",
    # "Ours less_trunc_2 (trimmed)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_trunc_2/ckpt_eval_surf_masked/ckpt/cf.txt",

}


df_all = None

digit = 2

for i, scene in enumerate(scene_names):
    df = get_scene_comp(scene, 1)
    df = df.round(digit)
    # df.loc[-1] = ["scene name", scene, scene]
    df = df.rename(columns={'Avg cf':scene})
    # df.loc[-1] = ["scene name", scene]
    # df.index = df.index + 1  # shifting index
    # df.sort_index(inplace=True)

    if i > 0:
        df = df.drop('name', axis=1)
    df_all = pd.concat([df_all, df], axis=1)

    

# df_all.loc['NeuS'] = ['neus nomask', 1.37, 1.21, 0.73, 0.40, 1.20, 0.70, 0.72, 1.01, 1.116, 0.82, 0.66, 1.69, 0.39, 0.49, 0.51]
df_all.loc['NeuS m'] = ['neus mask', 0.83, 0.98, 0.56, 0.37, 1.13, 0.59, 0.60, 1.45, 0.95, 0.78, 0.52, 1.43, 0.36, 0.45, 0.45]

# df_all.loc['HFS'] = ['HFS', 0.76, 1.32, 0.70, 0.39, 1.06, 0.63, 0.63, 1.15, 1.12, 0.80, 0.52, 1.22, 0.33, 0.49, 0.50]

scenes = [
#     "dtu_scan24", 
    "dtu_scan37",
    "dtu_scan40",
#     "dtu_scan55",
    "dtu_scan63", 
#     "dtu_scan65",
#     "dtu_scan83", 
    "dtu_scan69",
#     "dtu_scan97",
#     "dtu_scan105", 
#     "dtu_scan106",
    "dtu_scan110",
#     "dtu_scan114",
#     "dtu_scan118",
#     "dtu_scan122",
]

df_all = df_all[['name', *scenes]]

# df_all['mean cf'] = df_all['Avg cf'][1:].mean(axis=1).round(digit)
# df_all['avg'] = df_all[scenes].mean(axis=1).round(digit)
# df_all['mean F1'] = df_all['F1'][1:].mean(axis=1).round(digit)

df_all.style.apply(highlight_min_values)

,name,dtu_scan37,dtu_scan40,dtu_scan63,dtu_scan69,dtu_scan110
0,NeRF (sigma=10),1.900000,1.860000,1.860000,2.040000,1.960000
1,NeRF (sigma=50),1.460000,1.430000,1.660000,1.600000,1.750000
2,NeRF (sigma=100),1.340000,1.570000,2.990000,2.220000,2.430000
3,Ours less_trunc all surf,1.340000,1.360000,0.990000,1.910000,1.370000
NeuS m,neus mask,0.980000,0.560000,1.130000,1.450000,1.430000


In [64]:
print(df_all.to_latex(index=False))

\begin{tabular}{lrrrrr}
\toprule
                    name &  dtu\_scan37 &  dtu\_scan40 &  dtu\_scan63 &  dtu\_scan69 &  dtu\_scan110 \\
\midrule
         NeRF (sigma=10) &        1.90 &        1.86 &        1.86 &        2.04 &         1.96 \\
         NeRF (sigma=50) &        1.46 &        1.43 &        1.66 &        1.60 &         1.75 \\
        NeRF (sigma=100) &        1.34 &        1.57 &        2.99 &        2.22 &         2.43 \\
Ours less\_trunc all surf &        1.34 &        1.36 &        0.99 &        1.91 &         1.37 \\
               neus mask &        0.98 &        0.56 &        1.13 &        1.45 &         1.43 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_29719/1159970361.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_all.to_latex(index=False))


In [ ]:

\begin{tabular}{lrrrrr}
\toprule
                    name &  dtu\_scan37 &  dtu\_scan40 &  dtu\_scan63 &  dtu\_scan69 &  dtu\_scan110 \\
\midrule
         NeRF (sigma=10) &        1.90 &        1.86 &        1.86 &        2.04 &         1.96 \\
         NeRF (sigma=50) &        1.46 &        1.43 &        1.66 &        1.60 &         1.75 \\
        NeRF (sigma=100) &        1.34 &        1.57 &        2.99 &        2.22 &         2.43 \\
Ours                     &        1.34 &        1.36 &        0.99 &        1.91 &         1.37 \\
               neus mask &        0.98 &        0.56 &        1.13 &        1.45 &         1.43 \\
\bottomrule
\end{tabular}

In [11]:
df_all.loc['NeuS'] = ['neus', 1.37, 1.21, 0.73, 0.40, 1.20, 0.70, 0.72, 1.01, 1.116, 0.82, 0.66, 1.69, 0.39, 0.49, 0.51, 1]
df_all

,name,dtu_scan24,dtu_scan37,dtu_scan40,dtu_scan55,dtu_scan63,dtu_scan65,dtu_scan83,dtu_scan69,dtu_scan97,dtu_scan105,dtu_scan106,dtu_scan110,dtu_scan114,dtu_scan118,dtu_scan122,avg
0,NeRF (sigma=10),1.990,1.900,1.863,1.708,1.855,1.956,2.225,2.036,2.430,1.817,1.852,1.956,1.668,1.986,1.712,1.930
1,NeRF (sigma=50),1.375,1.460,1.430,1.218,1.663,1.848,2.026,1.600,1.978,1.263,1.485,1.745,1.452,1.280,1.211,1.536
2,NeRF (sigma=100),1.512,1.343,1.571,0.881,2.990,2.003,2.841,2.224,2.796,2.080,1.719,2.429,1.442,1.909,1.652,1.959
3,Ours less_sparse,1.690,1.842,1.562,1.212,1.301,2.192,1.695,2.550,3.862,1.595,1.785,1.654,1.452,2.470,1.618,1.899
4,Ours less_trunc,1.468,1.659,1.589,1.338,1.178,2.264,1.535,2.296,3.362,1.313,1.834,1.593,1.311,2.224,1.447,1.761
5,Ours less_trunc (trimmed),1.035,1.355,1.298,1.100,0.889,2.154,1.293,1.904,2.843,0.987,1.711,1.334,1.093,1.878,1.244,1.475
6,NeRF (depth 0.1),1.215,1.397,1.232,0.847,1.299,1.727,1.697,1.403,2.009,1.056,1.316,1.436,1.207,1.439,1.079,1.357
NeuS,neus,1.370,1.210,0.730,0.400,1.200,0.700,0.720,1.010,1.116,0.820,0.660,1.690,0.390,0.490,0.510,1.000


In [3]:
# abla

# models = {
    
#     r'Ours (w/o $\mathcal{L}_\delta$)': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla/solid2_no_tv/ckpt_eval_surf_single/ckpt/cf.txt",
#     # "Ours (solid2_no_tv/surf_masked)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla/solid2_no_tv/ckpt_eval_surf_masked/ckpt/cf.txt",
    
#     r'Ours (w/o $\mathcal{L}_\delta$, $\mathcal{L}_\mathbf{n}$)': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla/solid2_no_tv_no_norm/ckpt_eval_surf_single/ckpt/cf.txt",
#     # "Ours (solid2_no_tv_no_norm/surf_masked)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla/solid2_no_tv_no_norm/ckpt_eval_surf_masked/ckpt/cf.txt",
        
#     'Ours (w/o truncate)': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla/solid2_no_trunc/ckpt_eval_surf_single/ckpt/cf.txt",
#     # "Ours (solid2_no_trunc/surf_masked)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla/solid2_no_trunc/ckpt_eval_surf_masked/ckpt/cf.txt",
    
#     r'Ours (w/o $\mathcal{L}_c$)': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla/solid2_no_conv/ckpt_eval_surf_single/ckpt/cf.txt",
#     # "Ours (solid2_no_conv/surf_masked)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla/solid2_no_conv/ckpt_eval_surf_masked/ckpt/cf.txt",
    
#     'Ours (single lv)': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla/solid2_single_lv/ckpt_eval_surf_single/ckpt/cf.txt",
    
#     "Ours": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/solid_no_fs/ckpt_eval_surf_single/ckpt/cf.txt",
#     # "Ours (base/surf_masked)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/solid_no_fs/ckpt_eval_surf_masked/ckpt/cf.txt",
    
#     "Ours (base_trans/surf_single)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/trans_no_fs_low_sparse_2/ckpt_eval_surf_single/ckpt/cf.txt",
#     # "Ours (base_trans/surf_masked)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/trans_no_fs_low_sparse_2/ckpt_eval_surf_masked/ckpt/cf.txt",
    
#     "Ours (trans_no_tv/surf_single)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla/trans2_no_tv/ckpt_eval_surf_single/ckpt/cf.txt",
#     # "Ours (trans_no_tv/surf_masked)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla/trans2_no_tv/ckpt_eval_surf_masked/ckpt/cf.txt",
    
#     "Ours (trans_no_tv_no_norm/surf_single)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla/trans2_no_tv_no_norm/ckpt_eval_surf_single/ckpt/cf.txt",
#     # "Ours (trans_no_tv_no_norm/surf_masked)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla/trans2_no_tv_no_norm/ckpt_eval_surf_masked/ckpt/cf.txt",
    
#     "Ours (trans_no_trunc/surf_single)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla/trans2_no_trunc/ckpt_eval_surf_single/ckpt/cf.txt",
#     # "Ours (trans_no_trunc/surf_masked)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla/trans2_no_trunc/ckpt_eval_surf_masked/ckpt/cf.txt",
    
    

#     # "Ours (trans2_0001/surf)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/trans2_0001/ckpt_eval_surf/ckpt/cf.txt",
#     # "Ours (trans2_0001/surf_single)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/trans2_0001/ckpt_eval_surf_single/ckpt/cf.txt",
#     # "Ours (trans2_0001/surf_masked)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/trans2_0001/ckpt_eval_surf_masked/ckpt/cf.txt",
#     # "Ours (trans2_0001_no_sparse/surf_single)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/trans2_0001_no_sparse/ckpt_eval_surf_single/ckpt/cf.txt",
#     # "Ours (trans2_0001_no_sparse/surf_masked)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/trans2_0001_no_sparse/ckpt_eval_surf_masked/ckpt/cf.txt",
#     # "Ours (trans2_0001_no_sparse_no_entro/surf_single)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/trans2_0001_no_sparse_no_entro/ckpt_eval_surf_single/ckpt/cf.txt",
#     # "Ours (trans2_0001_no_sparse_no_entro/surf_masked)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/trans2_0001_no_sparse_no_entro/ckpt_eval_surf_masked/ckpt/cf.txt",

# }
models = {
    
    'no tv': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla_less_trunc/no_tv/ckpt_eval_surf_single/ckpt/cf.txt",
    
    'notvnorm': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla_less_trunc/no_tv_no_norm/ckpt_eval_surf_single/ckpt/cf.txt",
        
    'no trunc': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla_less_trunc/no_trunc/ckpt_eval_surf_single/ckpt/cf.txt",
    
    'no conv': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla_less_trunc/no_conv/ckpt_eval_surf_single/ckpt/cf.txt",
    
    'single lv': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla_less_trunc/single_lv/ckpt_eval_surf_single/ckpt/cf.txt",
    
    "Ours": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/solid_less_trunc/ckpt_eval_surf_single/ckpt/cf.txt",
    

}

scene_names = [
    # "ship_re", 
    "ficus",
    # "lego_re",
    "lyre",
    # "stair",
    # "glass_table",
    # "case",
    # "glasses2",
]

df_all = None

digit = 3

for i, scene in enumerate(scene_names):
    df = get_scene_comp(scene, 1e-2)
    df = df.round(digit)
    # df.loc[-1] = ["scene name", scene, scene]
    df = df.rename(columns={'Avg cf':scene})
    # df.loc[-1] = ["scene name", scene]
    # df.index = df.index + 1  # shifting index
    # df.sort_index(inplace=True)

    if i > 0:
        df = df.drop('name', axis=1)
    df_all = pd.concat([df_all, df], axis=1)

    

# df_all['mean cf'] = df_all['Avg cf'][1:].mean(axis=1).round(digit)
df_all['avg'] = df_all[scene_names].mean(axis=1).round(digit)
# df_all['mean F1'] = df_all['F1'][1:].mean(axis=1).round(digit)
df_all


,name,ficus,lyre,avg
0,no tv,0.828,0.586,0.707
1,notvnorm,1.121,0.702,0.912
2,no trunc,0.922,0.583,0.752
3,no conv,0.467,0.242,0.355
4,single lv,0.637,0.404,0.520
5,Ours,0.389,0.182,0.286


In [5]:
print(df_all.iloc[[0,1,2,3,4,5]].transpose().to_latex())

\begin{tabular}{lllllll}
\toprule
{} &      0 &         1 &         2 &        3 &          4 &      5 \\
\midrule
name  &  no tv &  notvnorm &  no trunc &  no conv &  single lv &   Ours \\
ficus &  0.828 &     1.121 &     0.922 &    0.467 &      0.637 &  0.389 \\
lyre  &  0.586 &     0.702 &     0.583 &    0.242 &      0.404 &  0.182 \\
avg   &  0.707 &     0.912 &     0.752 &    0.355 &       0.52 &  0.286 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_11627/3767710198.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_all.iloc[[0,1,2,3,4,5]].transpose().to_latex())


In [ ]:
\begin{tabular}{lllllll}
\toprule
{} &      0 &         1 &         2 &        3 &          4 &      5 \\
\midrule
name  &  no tv &  notvnorm &  no trunc &  no conv &  single lv &   Ours \\
ficus &  0.828 &     1.121 &     0.922 &    0.467 &      0.637 &  0.389 \\
lyre  &  0.586 &     0.702 &     0.583 &    0.242 &      0.404 &  0.182 \\
avg   &  0.707 &     0.912 &     0.752 &    0.355 &       0.52 &  0.286 \\
\bottomrule
\end{tabular}

In [13]:
scene_names = [
    "ship_re", 
    "lego_re", 
    "ficus",
    "drums",
    "mic_re",
    "materials",
    "hotdog_re",
    "chair_re",
]

df_all = None

digit = 3

for i, scene in enumerate(scene_names):
    df = get_scene_comp(scene, 1e-2)
    df = df.round(digit)
    # df.loc[-1] = ["scene name", scene, scene]
    df = df.rename(columns={'Avg cf':scene})
    # df.loc[-1] = ["scene name", scene]
    # df.index = df.index + 1  # shifting index
    # df.sort_index(inplace=True)

    if i > 0:
        df = df.drop('name', axis=1)
    df_all = pd.concat([df_all, df], axis=1)

    

# df_all['mean cf'] = df_all['Avg cf'][1:].mean(axis=1).round(digit)
df_all['avg'] = df_all[scene_names].mean(axis=1).round(digit)
# df_all['mean F1'] = df_all['F1'][1:].mean(axis=1).round(digit)
df_all.style.highlight_min(color = 'grey', axis = 0)

,name,ship_re,lego_re,ficus,drums,mic_re,materials,hotdog_re,chair_re,avg
0,NeRF (sigma=10),0.985000,0.726000,1.131000,1.763000,1.012000,0.945000,1.141000,1.991000,1.212000
1,NeRF (sigma=50),0.672000,0.528000,0.793000,1.460000,1.011000,0.947000,0.937000,1.201000,0.944000
2,NeRF (sigma=100),0.479000,0.703000,0.417000,1.048000,0.808000,1.025000,2.259000,1.077000,0.977000
3,Mip360 (sigma=10),1.094000,1.052000,1.457000,3.726000,1.156000,1.501000,1.953000,1.651000,1.699000
4,Mip360 (sigma=50),1.217000,0.704000,2.640000,2.301000,1.420000,2.474000,1.463000,2.224000,1.805000
5,Mip360 (sigma=100),1.412000,0.625000,2.883000,2.095000,1.394000,3.143000,1.672000,2.200000,1.928000
6,NeuS White,0.515000,0.671000,1.573000,3.483000,0.272000,0.168000,0.249000,0.226000,0.895000
7,NeuS,0.552000,0.619000,1.667000,10.273000,0.251000,0.210000,0.602000,0.210000,1.798000
8,HFS White,0.553000,0.534000,0.362000,5.499000,0.278000,0.272000,0.276000,0.321000,1.012000
9,HFS,0.514000,0.463000,0.374000,1.820000,0.274000,0.255000,0.272000,0.428000,0.550000


In [14]:
table = df_all.to_latex(index=False)



for k in replace_dict:
    table = table.replace(k, replace_dict[k])

print(table)

\begin{tabular}{lrrrrrrrrr}
\toprule
              name &  ship &  lego &  ficus &  drums &  mic &  materials &  hotdog &  chair &   avg \\
\midrule
   NeRF ($\sigma=10$) &    0.985 &    0.726 &  1.131 &  1.763 &   1.012 &      0.945 &      1.141 &     1.991 & 1.212 \\
   NeRF ($\sigma=50$) &    0.672 &    0.528 &  0.793 &  1.460 &   1.011 &      0.947 &      0.937 &     1.201 & 0.944 \\
  NeRF ($\sigma=100$) &    0.479 &    0.703 &  0.417 &  1.048 &   0.808 &      1.025 &      2.259 &     1.077 & 0.977 \\
 Mip360 ($\sigma=10$) &    1.094 &    1.052 &  1.457 &  3.726 &   1.156 &      1.501 &      1.953 &     1.651 & 1.699 \\
 Mip360 ($\sigma=50$) &    1.217 &    0.704 &  2.640 &  2.301 &   1.420 &      2.474 &      1.463 &     2.224 & 1.805 \\
Mip360 ($\sigma=100$) &    1.412 &    0.625 &  2.883 &  2.095 &   1.394 &      3.143 &      1.672 &     2.200 & 1.928 \\
        NeuS White &    0.515 &    0.671 &  1.573 &  3.483 &   0.272 &      0.168 &      0.249 &     0.226 & 0.895 \\
       

/tmp/ipykernel_26625/2380909454.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  table = df_all.to_latex(index=False)


In [16]:
\begin{tabular}{lrrrrrrrrr}
\toprule
              {} &  ship &  lego &  ficus &  drums &  mic &  materials &  hotdog &  chair &   avg \\
\midrule
   NeRF ($\sigma=10$) &    0.985 &    0.726 &  1.131 &  1.763 &   1.012 &      0.945 &      1.141 &     1.991 & 1.212 \\
   NeRF ($\sigma=50$) &    0.672 &    0.528 &  0.793 &  1.460 &   1.011 &      0.947 &      0.937 &     1.201 & 0.944 \\
  NeRF ($\sigma=100$) &    0.479 &    0.703 &  0.417 &  1.048 &   0.808 &      1.025 &      2.259 &     1.077 & 0.977 \\
 Mip360 ($\sigma=10$) &    1.094 &    1.052 &  1.457 &  3.726 &   1.156 &      1.501 &      1.953 &     1.651 & 1.699 \\
 Mip360 ($\sigma=50$) &    1.217 &    0.704 &  2.640 &  2.301 &   1.420 &      2.474 &      1.463 &     2.224 & 1.805 \\
Mip360 ($\sigma=100$) &    1.412 &    0.625 &  2.883 &  2.095 &   1.394 &      3.143 &      1.672 &     2.200 & 1.928 \\
              NeuS &    0.552 &    0.619 &  1.667 & 10.273 &   \first{}0.251 &      \first{}0.210 &     \second{} 0.602 &     \first{}0.210 & 1.798 \\
               HFS &    \second{}0.514 &    \first{}0.463 &  \first{}0.374 &  1.820 &   \second{}0.274 &      \second{}0.255 &      \first{}0.272 &     \second{}0.428 & \second{}0.550 \\
              Ours &    \first{}0.365 &    \second{}0.514 &  \second{}0.389 &  \first{}1.037 &   0.310 &      0.369 &      0.833 &     0.456 & \first{}0.534 \\
\midrule
  NeRF (depth 0.1) &    0.391 &    0.452 &  0.366 &  1.118 &   0.591 &      0.469 &      0.798 &     0.537 & 0.590 \\
Mip360 (depth 0.9) &    0.305 &    0.237 &  0.293 &  0.858 &   0.315 &      0.425 &      \firstalt{}0.496 &     \firstalt{}0.239 & \firstalt{}0.396 \\
Mip360 (depth 0.1) &    0.300 &    \firstalt{}0.236 &  1.716 & 12.553 &   0.305 &      0.413 &      0.611 &     0.241 & 2.047 \\
    Ours (trimmed) &    \firstalt{}0.277 &    0.376 &  \firstalt{}0.240 &  \firstalt{}0.617 &   \firstalt{}0.288 &      \firstalt{}0.317 &      0.702 &     0.349 & \firstalt{}0.396 \\
\bottomrule
\end{tabular}


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 5)

In [17]:
scene_names = [
    "lyre",
    "bee",
    "stair",
    "fence",
    "scale",
    "seat",
    "well",
    "slide",
    # "house",
    # "windwheel",
    # "swing",
    # "swing2",
    # "round",
    # "bridge",
]
df_all = None

digit = 3

for i, scene in enumerate(scene_names):
    df = get_scene_comp(scene, 1e-2)
    df = df.round(digit)
    # df.loc[-1] = ["scene name", scene, scene]
    df = df.rename(columns={'Avg cf':scene})
    # df.loc[-1] = ["scene name", scene]
    # df.index = df.index + 1  # shifting index
    # df.sort_index(inplace=True)

    if i > 0:
        df = df.drop('name', axis=1)
    df_all = pd.concat([df_all, df], axis=1)

    

# df_all['mean cf'] = df_all['Avg cf'][1:].mean(axis=1).round(digit)
df_all['avg'] = df_all[scene_names].mean(axis=1).round(digit)
# df_all['mean F1'] = df_all['F1'][1:].mean(axis=1).round(digit)
df_all.style.highlight_min(color = 'grey', axis = 0)

,name,lyre,bee,stair,fence,scale,seat,well,slide,avg
0,NeRF (sigma=10),0.740000,0.422000,0.265000,0.205000,0.793000,1.128000,0.608000,0.599000,0.595000
1,NeRF (sigma=50),0.805000,0.593000,0.208000,0.328000,0.959000,0.760000,0.708000,0.509000,0.609000
2,NeRF (sigma=100),0.543000,0.598000,0.212000,0.305000,0.884000,0.492000,0.703000,0.648000,0.548000
3,Mip360 (sigma=10),0.913000,0.803000,0.527000,0.864000,6.221000,1.286000,2.754000,0.999000,1.796000
4,Mip360 (sigma=50),1.311000,1.181000,0.279000,0.433000,2.243000,0.744000,1.941000,0.597000,1.091000
5,Mip360 (sigma=100),2.613000,2.252000,0.268000,0.445000,2.024000,0.541000,1.547000,0.526000,1.277000
6,NeuS White,0.819000,0.141000,4.019000,0.128000,0.242000,1.192000,0.357000,0.369000,0.908000
7,NeuS,0.812000,0.242000,4.087000,0.141000,0.237000,0.431000,0.360000,0.358000,0.834000
8,HFS White,0.817000,0.178000,4.191000,47.351000,nan,0.595000,0.325000,nan,8.910000
9,HFS,0.781000,0.336000,4.316000,nan,0.271000,0.425000,0.385000,nan,1.086000


In [18]:
table = df_all.to_latex(index=False)
for k in replace_dict:
    table = table.replace(k, replace_dict[k])

print(table)

\begin{tabular}{lrrrrrrrrr}
\toprule
              name &  lyre &   bee &  stair &  fence &  scale &  seat &  well &  slide &   avg \\
\midrule
   NeRF ($\sigma=10$) & 0.740 & 0.422 &  0.265 &  0.205 &  0.793 & 1.128 & 0.608 &  0.599 & 0.595 \\
   NeRF ($\sigma=50$) & 0.805 & 0.593 &  0.208 &  0.328 &  0.959 & 0.760 & 0.708 &  0.509 & 0.609 \\
  NeRF ($\sigma=100$) & 0.543 & 0.598 &  0.212 &  0.305 &  0.884 & 0.492 & 0.703 &  0.648 & 0.548 \\
 Mip360 ($\sigma=10$) & 0.913 & 0.803 &  0.527 &  0.864 &  6.221 & 1.286 & 2.754 &  0.999 & 1.796 \\
 Mip360 ($\sigma=50$) & 1.311 & 1.181 &  0.279 &  0.433 &  2.243 & 0.744 & 1.941 &  0.597 & 1.091 \\
Mip360 ($\sigma=100$) & 2.613 & 2.252 &  0.268 &  0.445 &  2.024 & 0.541 & 1.547 &  0.526 & 1.277 \\
        NeuS White & 0.819 & 0.141 &  4.019 &  0.128 &  0.242 & 1.192 & 0.357 &  0.369 & 0.908 \\
              NeuS & 0.812 & 0.242 &  4.087 &  0.141 &  0.237 & 0.431 & 0.360 &  0.358 & 0.834 \\
         HFS White & 0.817 & 0.178 &  4.191 & 47.351 &

/tmp/ipykernel_26625/1404706710.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  table = df_all.to_latex(index=False)


In [ ]:
\begin{tabular}{lrrrrrrrrr}
\toprule
              name &  lyre &   bee &  stair &  fence &  scale &  seat &  well &  slide &   avg \\
\midrule
   NeRF ($\sigma=10$) & 0.740 & 0.422 &  0.265 &  0.205 &  0.793 & 1.128 & 0.608 &  0.599 & 0.595 \\
   NeRF ($\sigma=50$) & 0.805 & 0.593 &  \second{}0.208 &  0.328 &  0.959 & 0.760 & 0.708 &  0.509 & 0.609 \\
  NeRF ($\sigma=100$) & \second{}0.543 & 0.598 &  0.212 &  0.305 &  0.884 & 0.492 & 0.703 &  0.648 & 0.548 \\
 Mip360 ($\sigma=10$) & 0.913 & 0.803 &  0.527 &  0.864 &  6.221 & 1.286 & 2.754 &  0.999 & 1.796 \\
 Mip360 ($\sigma=50$) & 1.311 & 1.181 &  0.279 &  0.433 &  2.243 & 0.744 & 1.941 &  0.597 & 1.091 \\
Mip360 ($\sigma=100$) & 2.613 & 2.252 &  0.268 &  0.445 &  2.024 & 0.541 & 1.547 &  0.526 & 1.277 \\
              NeuS    & 0.812 & \second{}0.242 &  4.087 &  \first{}0.141 &  \first{}0.237 & 0.431 & \first{}0.360 &  \first{}0.358 & 0.834 \\
               HFS    & 0.781 & 0.336 &  4.316 &    NaN &  0.271 & \first{}0.425 & 0.385 &    NaN & 1.086 \\
              Ours    & \first{}0.182 & \first{}0.229 &  \first{}0.184 &  \second{}0.155 &  0.641 & 0.536 & 0.442 &  0.558 & \first{0.366 \\
\midrule
  NeRF (depth 0.1)    & 0.416 & 0.351 &  0.215 &  0.263 &  0.788 & 0.495 & 0.500 &  0.498 & 0.441 \\
Mip360 (depth 0.9)    & 0.192 & 0.361 &  0.200 &  \firstalt{}0.139 &  0.741 & 0.270 & \firstalt{}0.210 &  0.448 & 0.320 \\
Mip360 (depth 0.1)    & \firstalt{}0.180 & 0.456 &  0.244 &  0.523 &  2.401 & \firstalt{}0.259 & 0.214 &  \firstalt{}0.426 & 0.588 \\
    Ours (trimmed)    & 0.188 & \firstalt{}0.207 &  \firstalt{}0.176 &  0.176 &  \firstalt{}0.575 & 0.288 & 0.319 &  0.506 & \firstalt{}0.304 \\
\bottomrule
\end{tabular}

In [19]:
scene_names = [
    # "bulb",
    # "crystal",
    # "pot",
    "glass_table",
    "jar",
    "glasses2",
    "wine",
    "lego_transparent",
    "bottle_2",
    "cup",
    "case",
    # "skull",
    # "vase", 
    # "perfume",
    # "bottle",
    # "stool",
    # "cupboard",
]
# "scale" "seat" "well" "house" "swing"
df_all = None

digit = 3

for i, scene in enumerate(scene_names):
    df = get_scene_comp(scene, 1e-2)
    df = df.round(digit)
    # df.loc[-1] = ["scene name", scene, scene]
    df = df.rename(columns={'Avg cf':scene})
    # df.loc[-1] = ["scene name", scene]
    # df.index = df.index + 1  # shifting index
    # df.sort_index(inplace=True)

    if i > 0:
        df = df.drop('name', axis=1)
    df_all = pd.concat([df_all, df], axis=1)

    

# df_all['mean cf'] = df_all['Avg cf'][1:].mean(axis=1).round(digit)
df_all['avg'] = df_all[scene_names].mean(axis=1).round(digit)
# df_all['mean F1'] = df_all['F1'][1:].mean(axis=1).round(digit)
df_all

,name,glass_table,jar,glasses2,wine,lego_transparent,bottle_2,cup,case,avg
0,NeRF (sigma=10),1.355,3.031,1.215,1.598,0.637,0.717,0.886,1.255,1.337
1,NeRF (sigma=50),0.989,2.943,0.871,2.050,0.628,0.929,1.141,1.200,1.344
2,NeRF (sigma=100),0.719,2.817,2.009,3.814,1.104,1.870,1.690,1.895,1.990
3,Mip360 (sigma=10),12.123,9.770,0.906,2.685,7.636,2.437,1.801,1.846,4.900
4,Mip360 (sigma=50),4.201,14.763,0.595,6.882,6.913,2.324,1.669,4.012,5.170
5,Mip360 (sigma=100),3.181,19.943,0.799,8.065,5.988,2.915,3.188,4.703,6.098
6,NeuS White,3.877,5.032,0.436,0.240,0.685,2.271,0.872,5.091,2.313
7,NeuS,3.979,2.170,0.195,0.464,NaN,NaN,NaN,NaN,1.702
8,HFS White,2.586,4.615,0.320,0.527,0.598,1.493,1.961,4.621,2.090
9,HFS,NaN,3.647,NaN,0.532,NaN,NaN,NaN,NaN,2.090


In [20]:
table = df_all.to_latex(index=False)
for k in replace_dict:
    table = table.replace(k, replace_dict[k])

print(table)

\begin{tabular}{lrrrrrrrrr}
\toprule
              name &  glass\_table &    jar &  glasses2 &  wine &  lego\_transparent &  bottle\_2 &   cup &  case &   avg \\
\midrule
   NeRF ($\sigma=10$) &        1.355 &  3.031 &     1.215 & 1.598 &             0.637 &     0.717 & 0.886 & 1.255 & 1.337 \\
   NeRF ($\sigma=50$) &        0.989 &  2.943 &     0.871 & 2.050 &             0.628 &     0.929 & 1.141 & 1.200 & 1.344 \\
  NeRF ($\sigma=100$) &        0.719 &  2.817 &     2.009 & 3.814 &             1.104 &     1.870 & 1.690 & 1.895 & 1.990 \\
 Mip360 ($\sigma=10$) &       12.123 &  9.770 &     0.906 & 2.685 &             7.636 &     2.437 & 1.801 & 1.846 & 4.900 \\
 Mip360 ($\sigma=50$) &        4.201 & 14.763 &     0.595 & 6.882 &             6.913 &     2.324 & 1.669 & 4.012 & 5.170 \\
Mip360 ($\sigma=100$) &        3.181 & 19.943 &     0.799 & 8.065 &             5.988 &     2.915 & 3.188 & 4.703 & 6.098 \\
        NeuS White &        3.877 &  5.032 &     0.436 & 0.240 &             0.

/tmp/ipykernel_26625/1404706710.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  table = df_all.to_latex(index=False)


In [46]:
\begin{tabular}{lrrrrrrrrr}
\toprule
              name &  glass table &    jar &  glasses &  wine &  lego t &  bottle &   cup &  case &   avg \\
\midrule
   NeRF ($\sigma=10$) &        1.355 &  3.031 &     1.215 & 1.598 &             0.637 &     \second{}0.717 & \second{}0.886 & 1.255 & \second{}1.337 \\
   NeRF ($\sigma=50$) &        0.989 &  2.943 &     0.871 & 2.050 &             0.628 &     0.929 & 1.141 & \second{}1.200 & 1.344 \\
  NeRF ($\sigma=100$) &        \first{}0.719 &  \second{}2.817 &     2.009 & 3.814 &             1.104 &     1.870 & 1.690 & 1.895 & 1.990 \\
 Mip360 ($\sigma=10$) &       12.123 &  9.770 &     0.906 & 2.685 &             7.636 &     2.437 & 1.801 & 1.846 & 4.900 \\
 Mip360 ($\sigma=50$) &        4.201 & 14.763 &     0.595 & 6.882 &             6.913 &     2.324 & 1.669 & 4.012 & 5.170 \\
Mip360 ($\sigma=100$) &        3.181 & 19.943 &     0.799 & 8.065 &             5.988 &     2.915 & 3.188 & 4.703 & 6.098 \\
        NeuS White    &        3.877 &  5.032 &     \second{}0.436 & \first{}0.240 &             0.685 &     2.271 & 0.872 & 5.091 & 2.313 \\
         HFS White    &        2.586 &  4.615 &     \first{}0.320 & 0.527 &             \second{}0.598 &     1.493 & 1.961 & 4.621 & 2.090 \\
              Ours    &        \second{}0.820 &  \first{}1.945 &     0.874 & \second{}0.351 &             \first{}0.534 &     \first{}0.688 & \first{}0.811 & \first{}0.721 & \first{}0.843 \\
\midrule
  NeRF (depth 0.1)    &        0.809 &  2.742 &     0.694 & 0.702 &             0.492 &     0.631 & 0.956 & 0.768 & 0.974 \\
Mip360 (depth 0.9)    &        4.070 &  2.383 &     \firstalt{}0.215 & 1.325 &             \firstalt{}0.387 &     1.323 & 0.792 & 0.816 & 1.414 \\
Mip360 (depth 0.1)    &        5.374 &  4.932 &     0.219 & 1.763 &             0.806 &     3.101 & 1.959 & 0.611 & 2.346 \\
    Ours (trimmed)    &        \firstalt{}0.670 &  \firstalt{}1.560 &     0.548 & \firstalt{}0.411 &             0.427 &     \firstalt{}0.604 & \firstalt{}0.711 & \firstalt{}0.510 & \firstalt{}0.680 \\
\bottomrule
\end{tabular}

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,scene name,NeRF (sig=10),NeRF (sig=50),NeRF (test mode 0.1),MipNeRF360 (sig=10),MipNeRF360 (sig=50),MipNeRF360 (test mode 0.9),Ours (clamp_0149_no_ldist_no_in_sparse/surf_ma...,Ours (clamp_0149_no_ldist_no_in_sparse/surf_si...,Ours (clamp_0149_mlv_2/surf_masked),Ours (clamp_0149_mlv_2/surf),NeuS 0.5,HFS 0.5
1,NaN,2.061,1.963,1.329,5.503,5.3,1.781,NaN,NaN,0.896,1.086,NaN,NaN
2,NaN,2.061,1.963,1.329,5.503,5.3,1.781,NaN,NaN,0.896,1.086,NaN,NaN


In [22]:
scene_names = [
    [   
    "ship_re", 
    "lego_re", 
    "ficus",
    "drums",
    "mic_re",
    "materials",
    "hotdog_re",
    "chair_re",
    ],
    [   
    "lyre",
    "bee",
    "stair",
    "fence",
    "scale",
    "seat",
    "well",
    "slide",
    ],
    [
    "glass_table",
    "jar",
    "glasses2",
    "wine",
    "lego_transparent",
    "bottle_2",
    "cup",
    "case",
    ]
]


group_names = ['NeRF Synthetic', 'Delicate', 'Semi-Transparent']

digit = 3

df_summary = None

for group_i in range(len(scene_names)):

    df_group = None
    for i, scene in enumerate(scene_names[group_i]):
        df = get_scene_comp(scene, 1e-2)
        df = df.round(digit)
        # df.loc[-1] = ["scene name", scene, scene]
        df.loc[-1] = ["scene name", scene]
        df.index = df.index + 1  # shifting index
        df.sort_index(inplace=True)

        if i > 0:
            df = df.drop('name', axis=1)
        df_group = pd.concat([df_group, df], axis=1)

        

    df_group[group_names[group_i]] = df_group['Avg cf'][1:].mean(axis=1).round(digit)
    # df_group['mean F1'] = df_group['F1'][1:].mean(axis=1).round(digit)
    df_group = df_group[['name', group_names[group_i]]].transpose()

    if df_summary is None:
        df_summary = df_group
    else:
        df_summary = pd.concat([df_summary, df_group.iloc[[1]]], axis=0)

df_summary = df_summary.drop(columns=[0]).transpose().set_index('name')
df_summary_original = df_summary.copy()
df_summary_original
# df_summary.at['Ours (clamp_0149_no_ldist_no_in_sparse/surf_masked)', 'name'] = 'Ours (Masked)'
# df_summary.at[9, 'name'] = 'Ours'

df_summary = df_summary.rename(index={
    'Ours (solid_no_fs/surf_single)' : 'Ours',
    'Ours (solid_no_fs/surf_masked)' : 'Ours (Masked)',
    'NeRF (test mode 0.1)' : 'NeRF (depth 0.1)',
    'MipNeRF360 (test mode 0.1)' : 'MipNeRF360 (depth 0.1)',
    'MipNeRF360 (test mode 0.9)' : 'MipNeRF360 (depth 0.9)',

})
# df_summary.at['Ours', group_names[2]] =  df_summary.at['Ours (trans_no_fs_low_sparse_2/surf_single)', group_names[2]]
# df_summary.at['Ours (trimmed)', group_names[2]] =  df_summary.at['Ours (trans_no_fs_low_sparse_2/surf_masked)', group_names[2]]
# df_summary.drop(['Ours (trans_no_fs_low_sparse_2/surf_single)', 'Ours (trans_no_fs_low_sparse_2/surf_masked)'], inplace=True)

df_summary.at['NeuS', group_names[2]] =  df_summary.at['NeuS White', group_names[2]]
df_summary.at['HFS', group_names[2]] =  df_summary.at['HFS White', group_names[2]]
df_summary.drop(['NeuS White', 'HFS White'], inplace=True)

df_summary
df_summary['average'] = df_summary.mean(axis=1).round(digit)
# print(df_summary.round(digit).to_latex())
df_summary.style.highlight_min(color = 'grey', axis = 0)

,NeRF Synthetic,Delicate,Semi-Transparent,average
name,,,,
NeRF (sigma=10),1.212000,0.595000,1.337000,1.048000
NeRF (sigma=50),0.944000,0.609000,1.344000,0.966000
NeRF (sigma=100),0.977000,0.548000,1.990000,1.172000
Mip360 (sigma=10),1.699000,1.796000,4.900000,2.798000
Mip360 (sigma=50),1.805000,1.091000,5.170000,2.689000
Mip360 (sigma=100),1.928000,1.277000,6.098000,3.101000
NeuS,1.798000,0.834000,2.313000,1.648000
HFS,0.550000,1.086000,2.090000,1.242000
Ours,0.534000,0.366000,0.843000,0.581000


In [23]:
table = df_summary.to_latex(index=True)
for k in replace_dict:
    table = table.replace(k, replace_dict[k])

print(table)

\begin{tabular}{llllr}
\toprule
{} & NeRF Synthetic & Delicate & Semi-Transparent &  average \\
name               &                &          &                  &          \\
\midrule
NeRF ($\sigma=10$)    &          1.212 &    0.595 &            1.337 &    1.048 \\
NeRF ($\sigma=50$)    &          0.944 &    0.609 &            1.344 &    0.966 \\
NeRF ($\sigma=100$)   &          0.977 &    0.548 &             1.99 &    1.172 \\
Mip360 ($\sigma=10$)  &          1.699 &    1.796 &              4.9 &    2.798 \\
Mip360 ($\sigma=50$)  &          1.805 &    1.091 &             5.17 &    2.689 \\
Mip360 ($\sigma=100$) &          1.928 &    1.277 &            6.098 &    3.101 \\
NeuS               &          1.798 &    0.834 &            2.313 &    1.648 \\
HFS                &           0.55 &    1.086 &             2.09 &    1.242 \\
Ours               &          0.534 &    0.366 &            0.843 &    0.581 \\
Ours (trimmed)     &          0.396 &    0.304 &             0.68 &    0.460 

/tmp/ipykernel_26625/2004402259.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  table = df_summary.to_latex(index=True)


In [120]:
\begin{tabular}{llllr}
\toprule
{} & Synthetic & Delicate & Semi-T &  Avg \\
\midrule
NeRF ($\sigma=10$)    &          1.212 &    0.595 &            \second{}1.337 &    1.048 \\
NeRF ($\sigma=50$)    &          0.944 &    0.609 &            1.344 &    \second{}0.966 \\
NeRF ($\sigma=100$)   &          0.977 &    \second{}0.548 &            1.990 &    1.172 \\
Mip360 ($\sigma=10$)  &          1.699 &    1.796 &            4.900 &    2.798 \\
Mip360 ($\sigma=50$)  &          1.805 &    1.091 &            5.170 &    2.689 \\
Mip360 ($\sigma=100$) &          1.928 &    1.277 &            6.098 &    3.101 \\
NeuS                  &          1.798 &    0.834 &            2.313 &    1.648 \\
HFS                   &          \second{}0.550 &    1.086 &            2.090 &    1.242 \\
Ours                  &         \first{}0.534 &   \first{}0.366 &           \first{}0.843 &   \first{}0.581 \\
\midrule
NeRF (depth 0.1)      &          0.590 &    0.441 &            0.974 &    0.668 \\
Mip360 (depth 0.9)    &          \firstalt{}0.396 &    0.320 &            1.414 &    0.710 \\
Mip360 (depth 0.1)    &          2.047 &    0.588 &            2.346 &    1.660 \\
Ours (trimmed)        &          \firstalt{}0.396 &    \firstalt{}0.304 &            \firstalt{}0.680 &    \firstalt{}0.460 \\
\bottomrule
\end{tabular}

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 5)

In [5]:
from pathlib import Path

scenes = Path('/rds/project/rds-qxpdOeYWi78/plenoxels/data/nerf_synthetic/').glob('*')

# scene_names = ['bee', 'chair']

max_pt = np.zeros(3)
min_pt = np.zeros(3)

n = ''

for scene in scenes:
    if (scene / 'shape.npy').exists():
        pt = np.load(str(scene / 'shape.npy'))
        if (pt > 1.5).any() or (pt < -1.5).any():
            print(scene)
        max_pt = np.stack([max_pt, pt.max(axis=0)]).max(axis=0)
        min_pt = np.stack([min_pt, pt.min(axis=0)]).min(axis=0)

print(max_pt)
print(min_pt)

[1.19984955 1.15110329 1.26621564]
[-1.25275354 -1.28758241 -1.1752893 ]


In [6]:
print(max_pt)
print(min_pt)

[1.19984955 1.15110329 1.26621564]
[-1.25275354 -1.28758241 -1.1752893 ]


In [35]:
pt = np.load('/rds/project/rds-qxpdOeYWi78/plenoxels/data/nerf_synthetic/lego_re/shape copy.npy')

In [7]:
pt = np.load('/rds/project/rds-qxpdOeYWi78/plenoxels/data/nerf_synthetic/swing/shape.npy')

pt.min()

-0.770674155509651

In [36]:
pt_mask = (pt < np.array([1.5, 1.5, 1.5])).all(axis=-1) & (pt > np.array([-1.5, -1.5, -1.5])).all(axis=-1)
# pt = pt[pt_mask]
pt_mask.shape

(8978171,)

In [37]:
pt[pt_mask].shape

(8688396, 3)

In [38]:
# np.save('/rds/project/rds-qxpdOeYWi78/plenoxels/data/nerf_synthetic/lego_re/shape.npy', pt[pt_mask])

In [11]:
min_pt

array([-1.25275354, -1.28758241, -1.1752893 ])

In [12]:
SCENE_NAME="vase"

df = pd.DataFrame()
for name in models.keys():
    e = read_entry(models[name].format(SCENE_NAME))
    e['name'] = name

    df = pd.concat([df, pd.DataFrame(e, index=[len(df)])])

df

,name,Avg cf,F1
0,NeRF (sig=10),22.047646,0.458595
1,NeRF (sig=50),20.286852,0.465120
2,NeRF (sig=100),19.201910,0.424388
3,NeRF (test mode 0.1),13.025085,0.611390
4,MipNeRF360 (test mode 0.9),14.974637,0.549973
5,MipNeRF360 (test mode 0.1),37.443050,0.428289
6,Ours (mlv_trunc_conv_2_0028_no_ldist/surf),10.982675,0.675963
7,Ours (clamp_0149_no_ldist/surf_masked),7.929710,0.740219
8,Ours (clamp_0149_no_ldist_no_in_sparse/surf_ma...,NaN,NaN
9,Ours (clamp_0149_mlv/surf_masked),8.376033,0.713553


In [13]:
SCENE_NAME="materials"

df = pd.DataFrame()
for name in models.keys():
    e = read_entry(models[name].format(SCENE_NAME))
    e['name'] = name

    df = pd.concat([df, pd.DataFrame(e, index=[len(df)])])

df

,name,Avg cf,F1
0,NeRF (sig=10),9.454893,0.710838
1,NeRF (sig=50),9.468854,0.723273
2,NeRF (sig=100),10.248624,0.725351
3,NeRF (test mode 0.1),4.686928,0.881806
4,MipNeRF360 (test mode 0.9),4.252188,0.933179
5,MipNeRF360 (test mode 0.1),4.130758,0.934760
6,Ours (mlv_trunc_conv_2_0028_no_ldist/surf),3.912256,0.913995
7,Ours (clamp_0149_no_ldist/surf_masked),3.091837,0.939701
8,Ours (clamp_0149_no_ldist_no_in_sparse/surf_ma...,3.088983,0.939760
9,Ours (clamp_0149_mlv/surf_masked),3.426816,0.925045


In [14]:
SCENE_NAME="chair_re"

df = pd.DataFrame()
for name in models.keys():
    e = read_entry(models[name].format(SCENE_NAME))
    e['name'] = name

    df = pd.concat([df, pd.DataFrame(e, index=[len(df)])])

df

,name,Avg cf,F1
0,NeRF (sig=10),19.912934,0.527325
1,NeRF (sig=50),12.007064,0.644485
2,NeRF (sig=100),10.767594,0.781235
3,NeRF (test mode 0.1),5.370772,0.864050
4,MipNeRF360 (test mode 0.9),2.390599,0.973529
5,MipNeRF360 (test mode 0.1),2.407728,0.973407
6,Ours (mlv_trunc_conv_2_0028_no_ldist/surf),4.446632,0.896778
7,Ours (clamp_0149_no_ldist/surf_masked),3.651723,0.922913
8,Ours (clamp_0149_no_ldist_no_in_sparse/surf_ma...,3.655053,0.922727
9,Ours (clamp_0149_mlv/surf_masked),3.495771,0.938290


In [15]:
SCENE_NAME="hotdog_re"

df = pd.DataFrame()
for name in models.keys():
    e = read_entry(models[name].format(SCENE_NAME))
    e['name'] = name

    df = pd.concat([df, pd.DataFrame(e, index=[len(df)])])

df

,name,Avg cf,F1
0,NeRF (sig=10),11.414013,0.688347
1,NeRF (sig=50),9.367275,0.663224
2,NeRF (sig=100),22.588624,0.457303
3,NeRF (test mode 0.1),7.981621,0.739634
4,MipNeRF360 (test mode 0.9),4.961343,0.859724
5,MipNeRF360 (test mode 0.1),6.106451,0.858172
6,Ours (mlv_trunc_conv_2_0028_no_ldist/surf),8.284622,0.757588
7,Ours (clamp_0149_no_ldist/surf_masked),6.975189,0.787235
8,Ours (clamp_0149_no_ldist_no_in_sparse/surf_ma...,6.974750,0.787258
9,Ours (clamp_0149_mlv/surf_masked),7.083933,0.768279


In [16]:
SCENE_NAME="mic_re"

df = pd.DataFrame()
for name in models.keys():
    e = read_entry(models[name].format(SCENE_NAME))
    e['name'] = name

    df = pd.concat([df, pd.DataFrame(e, index=[len(df)])])

df

,name,Avg cf,F1
0,NeRF (sig=10),10.115326,0.721858
1,NeRF (sig=50),10.107573,0.726430
2,NeRF (sig=100),8.083276,0.775938
3,NeRF (test mode 0.1),5.907683,0.857502
4,MipNeRF360 (test mode 0.9),3.149717,0.952480
5,MipNeRF360 (test mode 0.1),3.049015,0.955636
6,Ours (mlv_trunc_conv_2_0028_no_ldist/surf),3.436153,0.956512
7,Ours (clamp_0149_no_ldist/surf_masked),2.789582,0.970264
8,Ours (clamp_0149_no_ldist_no_in_sparse/surf_ma...,2.766675,0.970235
9,Ours (clamp_0149_mlv/surf_masked),3.376495,0.949894


In [17]:
SCENE_NAME="lyre"

df = pd.DataFrame()
for name in models.keys():
    e = read_entry(models[name].format(SCENE_NAME))
    e['name'] = name

    df = pd.concat([df, pd.DataFrame(e, index=[len(df)])])

df

,name,Avg cf,F1
0,NeRF (sig=10),7.403434,0.785264
1,NeRF (sig=50),8.048358,0.756796
2,NeRF (sig=100),5.425162,0.823886
3,NeRF (test mode 0.1),4.159080,0.913639
4,MipNeRF360 (test mode 0.9),1.917760,0.997077
5,MipNeRF360 (test mode 0.1),1.795103,0.995970
6,Ours (mlv_trunc_conv_2_0028_no_ldist/surf),1.785278,0.986406
7,Ours (clamp_0149_no_ldist/surf_masked),1.791950,0.990229
8,Ours (clamp_0149_no_ldist_no_in_sparse/surf_ma...,NaN,NaN
9,Ours (clamp_0149_mlv/surf_masked),NaN,NaN


In [18]:
SCENE_NAME="crystal"

df = pd.DataFrame()
for name in models.keys():
    e = read_entry(models[name].format(SCENE_NAME))
    e['name'] = name

    df = pd.concat([df, pd.DataFrame(e, index=[len(df)])])

df

,name,Avg cf,F1
0,NeRF (sig=10),45.441500,0.187217
1,NeRF (sig=50),9.021482,0.579743
2,NeRF (sig=100),272.387766,0.001684
3,NeRF (test mode 0.1),NaN,NaN
4,MipNeRF360 (test mode 0.9),NaN,NaN
5,MipNeRF360 (test mode 0.1),NaN,NaN
6,Ours (mlv_trunc_conv_2_0028_no_ldist/surf),NaN,NaN
7,Ours (clamp_0149_no_ldist/surf_masked),NaN,NaN
8,Ours (clamp_0149_no_ldist_no_in_sparse/surf_ma...,NaN,NaN
9,Ours (clamp_0149_mlv/surf_masked),NaN,NaN


In [19]:
SCENE_NAME="lotus"

df = pd.DataFrame()
for name in models.keys():
    e = read_entry(models[name].format(SCENE_NAME))
    e['name'] = name

    df = pd.concat([df, pd.DataFrame(e, index=[len(df)])])

df

,name,Avg cf,F1
0,NeRF (sig=10),9.859440,0.689937
1,NeRF (sig=50),5.992301,0.809394
2,NeRF (sig=100),6.076504,0.817451
3,NeRF (test mode 0.1),NaN,NaN
4,MipNeRF360 (test mode 0.9),NaN,NaN
5,MipNeRF360 (test mode 0.1),NaN,NaN
6,Ours (mlv_trunc_conv_2_0028_no_ldist/surf),NaN,NaN
7,Ours (clamp_0149_no_ldist/surf_masked),NaN,NaN
8,Ours (clamp_0149_no_ldist_no_in_sparse/surf_ma...,NaN,NaN
9,Ours (clamp_0149_mlv/surf_masked),NaN,NaN


In [20]:
SCENE_NAME="vase"

df = pd.DataFrame()
for name in models.keys():
    e = read_entry(models[name].format(SCENE_NAME))
    e['name'] = name

    df = pd.concat([df, pd.DataFrame(e, index=[len(df)])])

df

,name,Avg cf,F1
0,NeRF (sig=10),22.047646,0.458595
1,NeRF (sig=50),20.286852,0.465120
2,NeRF (sig=100),19.201910,0.424388
3,NeRF (test mode 0.1),13.025085,0.611390
4,MipNeRF360 (test mode 0.9),14.974637,0.549973
5,MipNeRF360 (test mode 0.1),37.443050,0.428289
6,Ours (mlv_trunc_conv_2_0028_no_ldist/surf),10.982675,0.675963
7,Ours (clamp_0149_no_ldist/surf_masked),7.929710,0.740219
8,Ours (clamp_0149_no_ldist_no_in_sparse/surf_ma...,NaN,NaN
9,Ours (clamp_0149_mlv/surf_masked),8.376033,0.713553


In [21]:
SCENE_NAME="maple"

df = pd.DataFrame()
for name in models.keys():
    e = read_entry(models[name].format(SCENE_NAME))
    e['name'] = name

    df = pd.concat([df, pd.DataFrame(e, index=[len(df)])])

df

,name,Avg cf,F1
0,NeRF (sig=10),3.254980,0.969001
1,NeRF (sig=50),2.857695,0.969782
2,NeRF (sig=100),2.874183,0.954422
3,NeRF (test mode 0.1),NaN,NaN
4,MipNeRF360 (test mode 0.9),NaN,NaN
5,MipNeRF360 (test mode 0.1),NaN,NaN
6,Ours (mlv_trunc_conv_2_0028_no_ldist/surf),NaN,NaN
7,Ours (clamp_0149_no_ldist/surf_masked),NaN,NaN
8,Ours (clamp_0149_no_ldist_no_in_sparse/surf_ma...,NaN,NaN
9,Ours (clamp_0149_mlv/surf_masked),NaN,NaN
